In [2]:
import bezier
from tkinter import *
import numpy as np
from PIL import ImageGrab

In [1]:
print(ord('\u4E00')-ord('\u9FFF'))

-20991


In [ ]:
#define global canvas and stuff
#canvasSize = 800

width = 1200
height = 850

mainBlue = "#5A837B"
lightBlue = "#B3CCC8"
offWhite = "#DADADA"

# Create object and size
root = Tk()
root.geometry( str(width) + "x" + str(height) )
root.configure(background=mainBlue)
#root.resizable(False, False) #make window not resize, this just makes things easier to not allow for an aadaptive layout

padding = 10
#top frame for options and character
topFrame = Frame(root, bg=mainBlue)
topFrame.pack(side='top', fill="both")
bottomFrame = Frame(root, bg=mainBlue)
bottomFrame.pack(side='bottom', fill="both", expand=True)

#code for the character frame in the top left
charFrame = Frame(topFrame, bg=lightBlue)
charFrame.pack(side='left',  fill='both', padx=padding,  pady=padding)

charLabel = Label(charFrame, bg=lightBlue, text="Symbol:", font= ('Arial 22'), padx=15)
charLabel.pack(side='left')
charEntryCode = Entry(charFrame, text="", font= ('Arial 22'), width=8)
charEntryCode.pack(side='left', padx=25)
charEntry = Entry(charFrame, text="", font= ('Arial 22'), width=4)
charEntry.pack(side='left', padx=15)

#code for the stroke frame in the bottom left
strokeFrame = Frame(bottomFrame, bg=lightBlue)
strokeFrame.pack(side='left',  fill='both', padx=padding,  pady=padding)

#weird configuration needed for scrollbar
scrollCanvas=Canvas(strokeFrame)
scrollFrame=Frame(scrollCanvas, background=lightBlue)
strokeScroll = Scrollbar(strokeFrame, orient="vertical", command=scrollCanvas.yview)
scrollCanvas.configure(yscrollcommand=strokeScroll.set)

strokeScroll.pack(side='right', fill='y')
scrollCanvas.pack(fill="both", expand=1, anchor="nw") 
wrapFrame = scrollCanvas.create_window((0,0), window=scrollFrame, anchor="nw")

def onFrameConfigure(canv):
    canv.configure(scrollregion=canv.bbox("all"))
    canv.itemconfigure(wrapFrame, width=canv.winfo_width())
scrollCanvas.bind("<Configure>", lambda event, canvas=scrollCanvas: onFrameConfigure(canvas))

strokeOptions = [*str("㇀㇁㇂㇃㇄㇅㇆㇇㇈㇉㇊㇋㇌㇍㇎㇏㇐㇑㇒㇓㇔㇕㇖㇗㇘㇙㇚㇛㇜㇝㇞㇟㇠㇡㇢㇣")] #list where each char is an entry
groupOptions = range(1,15) #list where each char is an entry

for i in range(1,10):
    strokeChoice = StringVar(value=strokeOptions[0])
    groupChoice = StringVar(value=groupOptions[0])
    
    strokeExampleFrame = Frame(scrollFrame, bg=offWhite)
    strokeExampleFrame.pack(side='top', fill='both', padx=24, pady=10) #abit strange this padx=24 is needed

    strokeExampleNameLabel = Label(strokeExampleFrame, text="Stroke #1", bg=offWhite, font=('Arial 14'))
    strokeExampleNameLabel.grid(row=0, column=0)

    strokeExampleDeleteButton = Button(strokeExampleFrame, text="X", font=('Arial 14'))
    strokeExampleDeleteButton.grid(row=0, column=4)

    strokeExampleTypeLabel = Label(strokeExampleFrame, text="Type:", bg=offWhite, font=('Arial 14'))
    strokeExampleTypeLabel.grid(row=1, column=0)
    strokeExampleTypeOptions = OptionMenu(strokeExampleFrame, strokeChoice, *strokeOptions)
    strokeExampleTypeOptions.config(width=8)
    strokeExampleTypeOptions.grid(row=1, column=1)

    strokeExampleGroupLabel = Label(strokeExampleFrame, text="Group:", bg=offWhite, font=('Arial 14'))
    strokeExampleGroupLabel.grid(row=2, column=0)
    strokeExampleGroupOptions = OptionMenu(strokeExampleFrame, groupChoice, *groupOptions)
    strokeExampleGroupOptions.config(width=4)
    strokeExampleGroupOptions.grid(row=2, column=1)

    strokeExampleDetectButton = Button(strokeExampleFrame, text="Detect", font=('Arial 14'))
    strokeExampleDetectButton.grid(row=2, column=2)
    strokeExampleEditButton = Button(strokeExampleFrame, text="Edit", font=('Arial 14'))
    strokeExampleEditButton.grid(row=2, column=3)
#add new stroke button
strokeNewButton = Button(scrollFrame, text="New Stroke", font=('Arial 18'))
strokeNewButton.pack(side='bottom', fill='both', padx=10, pady=10)





#code for the canvas frame in the bottom right
optionsFrame = Frame(topFrame, bg=mainBlue)
optionsFrame.pack(side='right',  fill='both',  padx=padding,  pady=padding,  expand=True)

autoButton = Button(optionsFrame, text="Auto-Gen", font=('Arial 14'))
autoButton.pack(side='left', padx=20, pady=10)
loadButton = Button(optionsFrame, text="Load", font=('Arial 14'))
loadButton.pack(side='left', padx=20, pady=10)
exportButton = Button(optionsFrame, text="Save & Export", font=('Arial 14'))
exportButton.pack(side='right', padx=20, pady=10)
saveButton = Button(optionsFrame, text="Save", font=('Arial 14'))
saveButton.pack(side='right', padx=20, pady=10)


#code for the option frame in the top right
pictureFrame = Frame(bottomFrame, bg=mainBlue)
pictureFrame.pack(side='left',  fill='both',  expand=True, padx=padding,  pady=padding)

root.update() #update the root so we can get width
canvasSize = min(pictureFrame.winfo_width(), pictureFrame.winfo_height())
canvas = Canvas(pictureFrame, bg="white", width=canvasSize, height=canvasSize) #our drawing board
canvas.pack(side="bottom")

#def resizeCanvas(self, event):
#    print("bruh")
#    size = min(pictureFrame.winfo_width(), pictureFrame.winfo_height())
#    canvas.config(width=size, height=self.size)
#canvas.bind("<Configure>", resizeCanvas)

#topFrame.pack( side = TOP )

#bottomFrame = Frame(root, bg="white", height=height - 100, width = width)
#bottomFrame.pack( side = BOTTOM )

#add label for draw char, label, text
#charLabel = Label(root, text = "Char: ")
#charLabel.pack()

#handle user typing in a char
#entryText = StringVar()
#charEntry = Entry(root, width = 10, justify="center", textvariable=entryText)
#entryText.set(drawChar) #default char
#charEntry.bind("<KeyPress>", onTextClear) #user is about to finish typing a char
#charEntry.bind("<KeyRelease>", onTextChange) #user entered a char
#charEntry.pack()

#button for processing the button
#processBtn = Button(root, text ="Gen. Strokes", command = saveCanvasAsImage)
#processBtn.pack()

#setup canvas
#canvas = Canvas(root, bg="white", height=canvasSize, width=canvasSize) #our drawing board
#canvas.create_text(400, 400, text=drawChar, font=('Microsoft YaHei','500'), fill='#555555')
#clickEventClear(0) #default arg of 0 as the event arg isn't used
#canvas.pack()

#reset the lists before we re-run the tkinter app
#currChar = []
#currStroke = []

#canvas.bind("<Button-1>", clickEventPoint) #setup the left click event, add a control point
#root.bind("<Key>", clickEventClear) #setup the left click event, clear the canvas
#canvas.bind("<Double-Button-1>", clickEventStroke) #setup the right click event, finalize stroke

root.mainloop() #run the app

In [117]:
import tkinter as tk
root = tk.Tk()
root.title("音樂編輯器")
root.geometry("600x480")

def onFrameConfigure(canvas):
    '''Reset the scroll region to encompass the inner frame'''
    canvas.configure(scrollregion=canvas.bbox("all"))
    '''When window size change, canvas size will change,
       use this line to change its item size (width).'''
    canvas.itemconfigure(wrapFrame, width=canvas.winfo_width())

canvas1 = tk.Canvas(root, highlightthickness=0)
frame = tk.Frame(canvas1, background="#FFFFFF")
vsb = tk.Scrollbar(root, orient="vertical", command=canvas1.yview)
canvas1.configure(yscrollcommand=vsb.set)

vsb.pack(side="right", fill="y")
canvas1.pack(fill="both", expand=1, anchor="nw") #canvas size is relative to window size.
wrapFrame = canvas1.create_window((0,0), window=frame, anchor="nw")

# When the window size change, it will call this function
def onFrameConfigure(canvas):
    '''Reset the scroll region to encompass the inner frame'''
    canvas.configure(scrollregion=canvas.bbox("all"))
    '''When window size change, canvas size will change,
       use this line to change its item size (width).'''
    canvas.itemconfigure(wrapFrame, width=canvas.winfo_width())
canvas1.bind("<Configure>", lambda event, canvas=canvas1: onFrameConfigure(canvas))

L1 = tk.Label(frame, text="音樂編輯器", bg="#556644", font=("",25))
L1.pack(anchor="n")

for i in range(100):
    input = tk.Entry(frame)
    input.pack()

root.mainloop()